In [5]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download("punkt")
import textblob 
from textblob import TextBlob
from textblob import Word
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\burak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\burak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\burak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Read Data

In [28]:
opmin_df = pd.read_csv("opmin.csv",usecols=['Text','Label'])
opmin_df


,Text,Label
0,So there is no way for me to plug it in here i...,0
1,Good case Excellent value.,1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2741,I think food should have flavor and texture an...,0
2742,Appetite instantly gone.,0
2743,Overall I was not impressed and would not go b...,0
2744,The whole experience was underwhelming and I t...,0


Text Preprocessing

In [29]:
opmin_series = opmin_df["Text"].apply(lambda x: " ".join(x.lower() for x in x.split())) #conversion of words to lowercase
opmin_series = opmin_series.str.replace("[^\w\s]","",regex= True) #deleting punctuation marks
opmin_series = opmin_series.str.replace("\d","",regex=True) #deleting numbers
opmin_text_df = pd.DataFrame(opmin_series, columns= ["Text"])
sw = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 
      'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 
      'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 
      'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
      'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 
      'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 
      'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain']
#I wanted to delete stopwords, but I especially wanted the words containing negativity to be in the data.
opmin_series = opmin_text_df["Text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw)) #deleting stopwords
opmin_text_df = pd.DataFrame(opmin_series, columns= ["Text"])
delete = pd.Series(" ".join(opmin_text_df["Text"]).split()).value_counts()[-3000:]
opmin_series = opmin_text_df["Text"].apply(lambda x: " ".join(x for x in x.split() if x not in delete)) #deleting rare wordsdeleting rare words
opmin_text_df = pd.DataFrame(opmin_series, columns= ["Text"])
opmin_series = opmin_text_df["Text"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))#lemmatization
opmin_text_df = pd.DataFrame(opmin_series, columns= ["Text"])
opmin_text_df

,Text
0,no way plug u unless go
1,good case excellent value
2,great jawbone
3,charger conversation lasting problem
4,mic great
...,...
2741,think food flavor texture lacking
2742,gone
2743,overall not impressed would not go back
2744,whole experience underwhelming think well go s...


Organised Data Set

In [38]:
opmin_df["Text"] = opmin_text_df
opmin_df["Label"].replace(0, value= "negative",inplace=True)
opmin_df["Label"].replace(1, value= "positive",inplace=True)
opmin_df.to_csv("org_opmin.csv")
df = pd.read_csv("org_opmin.csv",usecols=['Text','Label'], na_values=['','NA'])
df.isnull().sum() #We check whether there are any missing observations because we delete rare words.
df.dropna(inplace= True) #And we delete the NaN so that they don't cause us problems.
df.to_csv("org_opmin.csv") #Our edited data set is ready, we are done here.Let's move on to the machine learning section.